In [1]:
%pip install --upgrade pip
%pip install cmake

     ---------------------------------------- 1.8/1.8 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/37.6 MB ? eta -:--:--
   -- ------------------------------------- 2.4/37.6 MB 12.2 MB/s eta 0:00:03
   ----- ---------------------------------- 4.7/37.6 MB 11.9 MB/s eta 0:00:03
   ------- -------------------------------- 7.3/37.6 MB 11.9 MB/s eta 0:00:03
   ---------- ----------------------------- 9.7/37.6 MB 11.8 MB/s eta 0:00:03
   ------------- -------------------------- 12.3/37.6 MB 11.9 MB/s eta 0:00:03
   --------------- ------------------------ 14.7/37.6 MB 11.8 MB/s eta 0:00:02
   ------------------ --------------------- 17.3/37.6 MB 11.8 MB/s eta 0:00:02
   --------------------- ------------------ 19.9/37.6 MB 11.9 MB/s eta 0:00:02
   -----------

In [2]:
%pip install opencv-python opencv_contrib-python
%pip install face_recognition
%pip install numpy 

Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/100.1 MB ? eta -:--:--
      -------------------------------------- 2.4/100.1 MB 12.2 MB/s eta 0:00:09
     - ------------------------------------- 4.7/100.1 MB 11.9 MB/s eta 0:00:09
     -- ------------------------------------ 7.3/100.1 MB 11.9 MB/s eta 0:00:08
     --- ---------------------------------- 10.0/100.1 MB 11.9 MB/s eta 0:00:08
     ---- --------------------------------- 12.3/100.1 MB 11.9 MB/s eta 0:00:08
     ----- -------------------------------- 14.9/100.1 MB 11.9 MB/s eta 0:00:08
     ------ ------------------------------- 17.3/100.1 MB 11.8 MB/s eta 0:00:07
     ------- ------------------------------ 19.9/100.1 MB 11.9 MB/s eta 0:00:07
     -------- ----------------------------- 22.3/100.1 MB 11.8 MB/s eta 0:00:07
     --------- ---------------------------- 24.9/100.1 MB 11.9 MB/s eta 0:00:07
     ---------- --------------------------- 27.5/100.1 MB 11.

In [4]:
import cv2
import face_recognition
import os, numpy as np

In [6]:
#Directory to store training images
os.makedirs('training_faces', exist_ok=True)
os.makedirs('encodings', exist_ok=True)

Save faces

In [12]:

def capture_images():
    #Name for the captured images
    name = input("Enter name: ")

    #Create a folder for person if it doesnt exist yet
    person_folder = f"training_faces/{name}"
    os.makedirs(person_folder, exist_ok= True)


    #Initialize video capture
    cap = cv2.VideoCapture(0)

    #Frames Loop
    while True:
        success, frame = cap.read()
        if not success:
            print("Failed to capture frame")
            break

        cv2.imshow("Training - Press 'c' to capture, 'r' to restart or 'q' to quit", frame)

        #Capture the frame when 'c' is pressed
        if cv2.waitKey(1) & 0xFF == ord('c'):
            img_path = f'training_faces/{name}/{name}_{len(os.listdir(person_folder))+1}.jpg'
            cv2.imwrite(img_path, frame) #Save the original frame
            print(f"Image saved at: {img_path}")

        #Exit loop with when pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        if cv2.waitKey(1) & 0xFF ==ord('r'):
            cap.release()
            cv2.destroyAllWindows()
            return capture_images()

    cap.release()
    cv2.destroyAllWindows()


capture_images()


Image saved at: training_faces/Kiara/Kiara_1.jpg
Image saved at: training_faces/Kiara/Kiara_2.jpg
Image saved at: training_faces/Kiara/Kiara_3.jpg


Encode and train based on faces

In [13]:
def process_and_save_encodings():
    encodings = []
    names = []

    for person_name in os.listdir("training_faces"):
        person_folder = os.path.join("training_faces", person_name)
        if not os.path.isdir(person_folder):
            continue

        for img_file in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_file)
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            face_locs = face_recognition.face_locations(rgb_img)

            if len(face_locs) == 0:
                print(f"No face found in {img_file}, skipping.")
                continue

            encoding = face_recognition.face_encodings(rgb_img, known_face_locations=face_locs)[0]
            encodings.append(encoding)
            names.append(person_name)

    np.save("known_encodings.npy", encodings)
    np.save("known_names.npy", names)
    print(f"Saved {len(encodings)} face encodings for {len(set(names))} people.")

def load_encodings():
    if not os.path.exists("known_encodings.npy") or not os.path.exists("known_names.npy"):
        print("No encodings found, please run process_and_save_encodings() first.")
        return [], []
    encodings = np.load("known_encodings.npy", allow_pickle=True)
    names = np.load("known_names.npy", allow_pickle=True)
    return encodings, names


def start_recognition():
    known_encodings, known_names = load_encodings()
    if len(known_encodings) == 0:
        return

    cap = cv2.VideoCapture(0)
    while True:
        success, frame = cap.read()
        if not success:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encs = face_recognition.face_encodings(rgb_frame, face_locations)

        for (top, right, bottom, left), face_enc in zip(face_locations, face_encs):
            matches = face_recognition.compare_faces(known_encodings, face_enc, tolerance=0.5)
            name = "Unknown"

            if True in matches:
                matched_idxs = [i for i, val in enumerate(matches) if val]
                counts = {}
                for idx in matched_idxs:
                    counts[known_names[idx]] = counts.get(known_names[idx], 0) + 1
                name = max(counts, key=counts.get)

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        cv2.imshow("Real-Time Face Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

In [14]:
process_and_save_encodings()

No face found in Kiara_3.jpg, skipping.
Saved 8 face encodings for 3 people.


In [15]:
start_recognition()